In [3]:
import os
import requests
import sys
from service.Classifier.DataLoader.DataLoader import DataLoader
from service.Classifier.Enums.priority import Priority
from service.Classifier.DataLoader.P2DataLoader import P2DataLoader
from service.Classifier.DataLoader.P3DataLoader import P3DataLoader
from service.Classifier.DataLoader.P4DataLoader import P4DataLoader
from service.Classifier.Model.XGBRegressionModel import XGBRegressionModel
from service.Classifier.PreProcessing.RegressionModelPreProcessor import RegressionModelPreProcessor
import pickle
import platform
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import csv
from os import path
import pathlib
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report, f1_score, precision_score, recall_score
import csv
import os.path
from os import path
import pathlib
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from service.Classifier.Enums.LabelEnum import LabelEnum
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from service.Classifier.PreProcessing.Utils.dataValidator import DataValidator
from xgboost import XGBRegressor
from sklearn.pipeline import FeatureUnion
from service.Classifier.Enums.LabelEnum import LabelEnum
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from service.Classifier.PreProcessing.Utils.dataValidator import DataValidator
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

print('Python version:', platform.python_version())
print('tensorflow version:', tf.__version__)
sys.path.insert(0, os.path.abspath('../'))

Python version: 3.8.5
tensorflow version: 2.5.0


/Users/fjirigesi/Documents/eBay/OOSLAClassifier/service/Classifier/Model/XGBRegressionModel.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  from pandas import np


In [4]:
def initializeByPriority(priority):
    # load new updated data via Features class
    if priority == Priority.P2.value:
        return P2DataLoader()
    elif priority == Priority.P3.value:
        return P3DataLoader()
    elif priority == Priority.P4.value:
        return P4DataLoader()
    return None

In [5]:
from service.Classifier.DataSplit.TrainTestSplit import TrainTestSplit
from service.Classifier.Enums.dataTypeEnum import DataType

def preprocessData(priority_value):

    dataLoader = initializeByPriority(priority)
    data_df = dataLoader.loadTrainingRawData()

    # split data into train test 4:1
    dataSplit = TrainTestSplit()
    train_df, test_df = dataSplit.split(data_df)

    # generate features
    train_df, feature_names = dataLoader.transformRawDataToFeatures(train_df, DataType.TRAINDATA.value)
    test_df, feature_names = dataLoader.transformRawDataToFeatures(test_df, DataType.VALIDATION.value)
    print(train_df.shape)
    print(test_df.shape)

    # pre-processing the data based on model type
    preprocessor = RegressionModelPreProcessor(feature_names)
    train_X, train_y, test_X, test_y = preprocessor.preprocessing(train_df, test_df)
    print(train_X.shape)
    print(test_X.shape)
    
    return data_df, train_df, test_df, train_X, train_y, test_X, test_y

In [6]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold

def getGCX():
    url = "http://paysplan.paysplan-ns.svc.33.tess.io/people/gcx/allNames"
    return requests.get(url).json()

def checkGCX(row):
    if row['Submitter'] in GCX['people']:
        return 'Internal'
    else:
        return 'External'

def extractLatestCategory(row):
    try:
        row['Category'][-1]['value']
        return row['Category'][-1]['value']
    except:
        return 'NAN'

def getStaticMetrics(data_df):
    staticMetrics = data_df[["_id", "assignee", "reporter","issueType", "summary"]].copy()
    
    staticMetrics = staticMetrics.rename({'_id': "issueKey",'assignee': 'Owner', "issueType": 'Type', 
                                          'reporter': 'Submitter'}, axis='columns')
    
    staticMetrics['ESC'] = staticMetrics.apply(checkGCX, axis=1)
    
    with open('ExtarctedInfo.pickle', 'rb') as f:
        allInfor = pickle.load(f)
        
    # Join the table 
    right = allInfor[["issueKey","categoryBugs", "severity" ]]
    StaticMetricsResult = pd.merge(staticMetrics, right, how="left", on="issueKey")
    
    StaticMetricsResult = StaticMetricsResult.rename({'categoryBugs': 'Category', 'severity': 'Severity', 'summary': 'Summary'}, axis='columns')
    return StaticMetricsResult

def embeddingValues(col_name, staticMetrics):
    List = staticMetrics[col_name].tolist()
    uniqueListA = list(set(List))
    
    def getIndex(row):
        return uniqueListA.index(row[col_name])

    new_col_name = col_name + '_Emb'
    staticMetrics[new_col_name] = staticMetrics.apply(getIndex, axis=1)
    


In [7]:
class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [8]:
Summary = Pipeline([
                ('selector', TextSelector(key='Summary')),
                ('tfidf', TfidfVectorizer(stop_words='english'))
            ])

Owner_Emb =  Pipeline([
                ('selector', NumberSelector(key='Owner_Emb')),
                ('standard', StandardScaler())
            ])
Submitter_Emb =  Pipeline([
                ('selector', NumberSelector(key='Submitter_Emb')),
                ('standard', StandardScaler())
            ])
ESC_Emb =  Pipeline([
                ('selector', NumberSelector(key='ESC_Emb')),
                ('standard', StandardScaler())
            ])
Type_Emb =  Pipeline([
                ('selector', NumberSelector(key='Type_Emb')),
                ('standard', StandardScaler()),
            ])

Severity_Emb =  Pipeline([
                ('selector', NumberSelector(key='Severity_Emb')),
                ('standard', StandardScaler()),
            ])
Category_Emb =  Pipeline([
                ('selector', NumberSelector(key='Category_Emb')),
                ('standard', StandardScaler()),
            ])

In [31]:
def valid(submission, priority):
    coefficients_Dict = {"P1": 1, "P2": 1, "P3": 1, "ALL": 1}
    coefficients = coefficients_Dict[priority]
    priorityDays_dict = {"P1": 20, "P2": 60, "P3": 90}
    daysAllowedList = priorityDays_dict[priority]
    
    submission["prediction"] = pd.Series(submission["ClosedDay"] >= coefficients * daysAllowedList,
                                         index=submission.index)
    
    submission["truth"] = pd.Series(submission["Actual"] >= daysAllowedList, index=submission.index)
    # submission["issueType"] = test_df["issueType"]
    return submission

In [32]:
from xgboost import XGBRegressor
from sklearn.pipeline import FeatureUnion
from service.Classifier.Enums.LabelEnum import LabelEnum
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from service.Classifier.PreProcessing.Utils.dataValidator import DataValidator
from xgboost import XGBRegressor


feats = FeatureUnion([('Owner_Emb', Owner_Emb), 
                      ('Submitter_Emb', Submitter_Emb),
                      ('ESC_Emb', ESC_Emb),
                      ('Type_Emb', Type_Emb),
                      ('Severity_Emb', Severity_Emb),
                      ('Category_Emb', Category_Emb),
                      ('Summary', Summary)])

feature_processing = Pipeline([('feats', feats)])


def evaluate_static_result(staticMetrics_trainX, staticMetrics_trainy, staticMetrics_testX, staticMetrics_testy):

    feature_processing.fit_transform(staticMetrics_trainX)

    pipeline = Pipeline([
        ('features',feats),
        ('regressor', XGBRegressor(max_depth=5, n_estimators=100, learning_rate=0.05)),
    ])


    pipeline.fit(staticMetrics_trainX, staticMetrics_trainy)

    predictions = np.rint(pipeline.predict(staticMetrics_testX)).astype(np.int64)


    submission = pd.DataFrame({'ClosedDay': predictions, 'Actual': staticMetrics_testy['ResolvedDay'].tolist()})

    rmse = np.sqrt(mean_squared_error(predictions, staticMetrics_testy))

    mae = mean_absolute_error(predictions, staticMetrics_testy)

    valid(submission, priority)

    f1 = f1_score(submission["truth"], submission["prediction"])
    precision = precision_score(submission["truth"], submission["prediction"])
    recall = recall_score(submission["truth"], submission["prediction"])


    print('rmse:', rmse)
    print('mae:', mae)
    print(confusion_matrix(submission["truth"], submission["prediction"]))
    print(classification_report(submission["truth"], submission["prediction"]))
    
    return (staticMetrics_testy, predictions, submission["truth"], submission["prediction"])

In [33]:
def extract_static_feature(priority):
    
    data_df, train_df,test_df, train_X, train_y, test_X, test_y = preprocessData(priority)

    

    staticMetrics = getStaticMetrics(data_df)
    
    staticMetrics['Category'] = staticMetrics.apply(extractLatestCategory, axis=1)

    # """
    # Embedding all columns except "Summary"
    # """

    colNames = ["Owner", "Submitter", "Type", "ESC", "Category", "Severity"]
    
    for colName in colNames:
        embeddingValues(colName, staticMetrics)

    staticMetrics = staticMetrics.rename({"issueKey": "keyID"}, axis='columns')
    staticMetrics = staticMetrics.drop_duplicates(subset='keyID', keep="last")
    staticMetrics["keyID"].is_unique

    AllMetrics_train = pd.merge(train_df, staticMetrics, how="left", on="keyID")
    AllMetrics_test = pd.merge(test_df, staticMetrics, how="left", on="keyID")


    # get training data 
    staticMetrics_trainX = AllMetrics_train[["Owner_Emb", "Submitter_Emb", "ESC_Emb", "Type_Emb", "Severity_Emb", "Category_Emb", "Summary"]]
    staticMetrics_trainy = AllMetrics_train[["ResolvedDay"]]
    # get testing data 
    staticMetrics_testX = AllMetrics_test[["Owner_Emb", "Submitter_Emb", "ESC_Emb", "Type_Emb", "Severity_Emb", "Category_Emb", "Summary"]]
    staticMetrics_testy = AllMetrics_test[["ResolvedDay"]]

    return (staticMetrics_trainX, staticMetrics_trainy, staticMetrics_testX, staticMetrics_testy)

# static_X=np.concatenate((staticMetrics_trainX, staticMetrics_testX))
# static_y = np.concatenate((staticMetrics_trainy, staticMetrics_testy))
# print(static_X.shape)
# print(static_y.shape)

## Get P1 Static Results

In [34]:
priority = Priority.P2.value
GCX = getGCX()
staticMetrics_trainX, staticMetrics_trainy, staticMetrics_testX, staticMetrics_testy = extract_static_feature(priority)


/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: PX
The project is not in embedding: CSCNPLAT
The project is not in embedding: GGRIP
The project is not in embedding: PROBLEM
The project is not in embedding: NATBEMCR
(2648, 12)
(646, 12)
(2648, 8)
(646, 8)


#### Extracted featrues 

In [35]:
staticMetrics_trainX.head(10)

,Owner_Emb,Submitter_Emb,ESC_Emb,Type_Emb,Severity_Emb,Category_Emb,Summary
0,1667,696,1,0,1,1,[Adyen Issue] DE - MP Unable to verify account...
1,382,142,0,0,2,4,Seller cannot cancel item from ResolutionCenter
2,1938,1213,0,0,2,10,Top of funnel for B2C registration (PPA) has d...
3,188,1419,0,0,0,0,US UK - Email sent to members in German langua...
4,1434,353,1,0,0,0,MP Account still charged Add Rate Fee's for C...
5,1049,1051,1,0,2,4,MP: Member's payouts on hold due to USER_PAYOU...
6,1802,1350,0,0,0,0,GCX unable to check Job Statuses - CSReadJobs ...
7,1667,1051,1,0,2,3,[Adyen Issue] MP: Possible Data Transmission I...
8,1563,353,1,0,0,0,MP - US - Member was charged for 2 labels but ...
9,1118,1336,1,0,2,16,CLONE - ALERT12866 - Global - Member unable to...


In [36]:
staticMetrics_trainX.to_csv("static.csv")

# results

## P1 reports

In [13]:
p1_regression_true, p1_regression_pred, p1_classification_true, p1_classification_pred = evaluate_static_result(staticMetrics_trainX, staticMetrics_trainy, staticMetrics_testX, staticMetrics_testy)

rmse: 9.442112495204292
mae: 7.671317829457364
[[ 80 307]
 [ 28 230]]
              precision    recall  f1-score   support

       False       0.74      0.21      0.32       387
        True       0.43      0.89      0.58       258

    accuracy                           0.48       645
   macro avg       0.58      0.55      0.45       645
weighted avg       0.62      0.48      0.43       645



## Get P2 Static Results

In [11]:
priority = Priority.P2.value

staticMetrics_trainX, staticMetrics_trainy, staticMetrics_testX, staticMetrics_testy = extract_static_feature(priority)

p2_regression_true, p2_regression_pred, p2_classification_true, p2_classification_pred = evaluate_static_result(staticMetrics_trainX, staticMetrics_trainy, staticMetrics_testX, staticMetrics_testy)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


(2740, 16)
(668, 16)
(2740, 12)
(668, 12)
rmse: 19.395775513869797
mae: 14.154191616766466
[[421  61]
 [135  51]]
              precision    recall  f1-score   support

       False       0.76      0.87      0.81       482
        True       0.46      0.27      0.34       186

    accuracy                           0.71       668
   macro avg       0.61      0.57      0.58       668
weighted avg       0.67      0.71      0.68       668



## Get P3 Static Results

In [12]:
priority = Priority.P3.value

staticMetrics_trainX, staticMetrics_trainy, staticMetrics_testX, staticMetrics_testy = extract_static_feature(priority)

p3_regression_true, p3_regression_pred, p3_classification_true, p3_classification_pred = evaluate_static_result(staticMetrics_trainX, staticMetrics_trainy, staticMetrics_testX, staticMetrics_testy)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


(1563, 16)
(382, 16)
(1563, 12)
(382, 12)
rmse: 25.426827568720093
mae: 17.12041884816754
[[266   8]
 [ 95  13]]
              precision    recall  f1-score   support

       False       0.74      0.97      0.84       274
        True       0.62      0.12      0.20       108

    accuracy                           0.73       382
   macro avg       0.68      0.55      0.52       382
weighted avg       0.70      0.73      0.66       382

